<a href="https://colab.research.google.com/github/JoelAbadi/DI-Bootcamp/blob/main/mini_proyect_XP_W7_D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()


Saving product_category_name_translation.csv to product_category_name_translation.csv
Saving olist_sellers_dataset.csv to olist_sellers_dataset.csv
Saving olist_products_dataset.csv.zip to olist_products_dataset.csv.zip
Saving olist_orders_dataset.csv.zip to olist_orders_dataset.csv.zip
Saving olist_order_reviews_dataset.csv.zip to olist_order_reviews_dataset.csv.zip
Saving olist_order_payments_dataset.csv.zip to olist_order_payments_dataset.csv.zip
Saving olist_geolocation_dataset.csv.zip to olist_geolocation_dataset.csv.zip
Saving olist_order_items_dataset.csv.zip to olist_order_items_dataset.csv.zip
Saving olist_customers_dataset.csv.zip to olist_customers_dataset.csv.zip


In [5]:
import os
os.listdir()


['.config',
 'olist_order_payments_dataset.csv.zip',
 'product_category_name_translation.csv',
 'olist_sellers_dataset.csv',
 'olist_customers_dataset.csv.zip',
 'olist_products_dataset.csv.zip',
 'brazilian-e-commerce-dive-deep-using-sql (1).ipynb',
 'olist_orders_dataset.csv.zip',
 'olist_geolocation_dataset.csv.zip',
 'olist_order_reviews_dataset.csv.zip',
 'olist_order_items_dataset.csv.zip',
 'brazilian-e-commerce-dive-deep-using-sql.ipynb',
 'sample_data']

In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine


In [7]:
df_customers = pd.read_csv('olist_customers_dataset.csv')
df_orders = pd.read_csv('olist_orders_dataset.csv')
df_order_items = pd.read_csv('olist_order_items_dataset.csv')
df_order_payments = pd.read_csv('olist_order_payments_dataset.csv')
df_order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
df_sellers = pd.read_csv('olist_sellers_dataset.csv')
df_products = pd.read_csv('olist_products_dataset.csv')
df_categories = pd.read_csv('product_category_name_translation.csv')
df_geo = pd.read_csv('olist_geolocation_dataset.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'olist_customers_dataset.csv'

In [8]:
import zipfile

zip_files = [f for f in os.listdir() if f.endswith('.zip')]

for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall('.')

In [9]:
df_customers = pd.read_csv('olist_customers_dataset.csv')
df_orders = pd.read_csv('olist_orders_dataset.csv')
df_order_items = pd.read_csv('olist_order_items_dataset.csv')
df_order_payments = pd.read_csv('olist_order_payments_dataset.csv')
df_order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
df_sellers = pd.read_csv('olist_sellers_dataset.csv')
df_products = pd.read_csv('olist_products_dataset.csv')
df_categories = pd.read_csv('product_category_name_translation.csv')
df_geo = pd.read_csv('olist_geolocation_dataset.csv')

In [10]:
engine = create_engine('sqlite://', echo=False)

df_customers.to_sql("olist_customers", con=engine, index=False)
df_orders.to_sql("olist_orders", con=engine, index=False)
df_order_items.to_sql("olist_order_items", con=engine, index=False)
df_order_payments.to_sql("olist_order_payments", con=engine, index=False)
df_order_reviews.to_sql("olist_order_reviews", con=engine, index=False)
df_sellers.to_sql("olist_sellers", con=engine, index=False)
df_products.to_sql("olist_products_dataset", con=engine, index=False)
df_categories.to_sql("product_category_name_translation", con=engine, index=False)
df_geo.to_sql("olist_geolocation", con=engine, index=False)


1000163

In [11]:
sql = '''
SELECT * FROM olist_customers
LIMIT 5
'''

df_result = pd.read_sql_query(sql, con=engine)
df_result.head()


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


Query 1: Count and Percentage of Orders in Jan 2018 with 5-Star Review

In [16]:
sql = """
SELECT
    COUNT(*) AS five_star_orders,
    100.0 * COUNT(*) / (
        SELECT COUNT(*)
        FROM olist_orders
        WHERE strftime('%Y-%m', order_purchase_timestamp) = '2018-01'
    ) AS percent_of_jan2018
FROM olist_orders o
JOIN olist_order_reviews r ON o.order_id = r.order_id
WHERE strftime('%Y-%m', o.order_purchase_timestamp) = '2018-01'
  AND r.review_score = 5
"""

df_query1 = pd.read_sql_query(sql, con=engine)
df_query1.head()


,five_star_orders,percent_of_jan2018
0,4097,56.362636


Query 2: Customer Purchase Trend Year on Year

In [15]:
sql = """
SELECT
    strftime('%Y', order_purchase_timestamp) AS year,
    COUNT(*) AS total_orders
FROM olist_orders
GROUP BY year
ORDER BY year
"""

df_query2 = pd.read_sql_query(sql, con=engine)
df_query2.head()


,year,total_orders
0,2016,329
1,2017,45101
2,2018,54011


Query 3: Average Order Values of Customers

In [17]:
sql = """
SELECT
    c.customer_unique_id,
    ROUND(AVG(order_total), 2) AS avg_order_value
FROM (
    SELECT o.customer_id, SUM(p.payment_value) AS order_total
    FROM olist_orders o
    JOIN olist_order_payments p ON o.order_id = p.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY o.order_id, o.customer_id
) AS order_totals
JOIN olist_customers c ON order_totals.customer_id = c.customer_id
GROUP BY c.customer_unique_id
ORDER BY avg_order_value DESC
"""

df_query3 = pd.read_sql_query(sql, con=engine)
df_query3.head()


,customer_unique_id,avg_order_value
0,0a0a92112bd4c708ca5fde585afaa872,13664.08
1,763c8b1c9c68a0229c42c9fc6f662b93,7274.88
2,dc4802a71eae9be1dd28f5d788ceb526,6929.31
3,459bef486812aa25204be022145caa62,6922.21
4,ff4159b92c40ebe40454e3e6a7c35ed6,6726.66


Query 4: Top 5 Cities with Highest Revenue (2016 to 2018)

In [18]:
sql = """
SELECT
    c.customer_city AS city,
    ROUND(SUM(p.payment_value), 2) AS total_revenue
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
JOIN olist_order_payments p ON o.order_id = p.order_id
WHERE o.order_status = 'delivered'
  AND o.order_purchase_timestamp BETWEEN '2016-01-01' AND '2018-12-31'
GROUP BY city
ORDER BY total_revenue DESC
LIMIT 5
"""

df_query4 = pd.read_sql_query(sql, con=engine)
df_query4.head()


,city,total_revenue
0,sao paulo,2108162.22
1,rio de janeiro,1111872.32
2,belo horizonte,406050.87
3,brasilia,345221.87
4,curitiba,238582.26


Query 5: State-Wise Revenue Table Between 2016 to 2018

In [19]:
sql = """
SELECT
    c.customer_state AS state,
    ROUND(SUM(p.payment_value), 2) AS total_revenue_2016_18
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
JOIN olist_order_payments p ON o.order_id = p.order_id
WHERE o.order_status = 'delivered'
  AND o.order_purchase_timestamp BETWEEN '2016-01-01' AND '2018-12-31'
GROUP BY state
ORDER BY total_revenue_2016_18 DESC
"""

df_query5 = pd.read_sql_query(sql, con=engine)
df_query5.head()


,state,total_revenue_2016_18
0,SP,5770266.19
1,RJ,2055690.45
2,MG,1819277.61
3,RS,861802.40
4,PR,781919.55


Query 6.1: Top Successful Sellers in Terms of Goods Sold

In [20]:
sql = """
SELECT
    seller_id,
    COUNT(*) AS items_sold
FROM olist_order_items
GROUP BY seller_id
ORDER BY items_sold DESC
LIMIT 5
"""

df_query6a = pd.read_sql_query(sql, con=engine)
df_query6a.head()


,seller_id,items_sold
0,6560211a19b47992c3666cc44a7e94c0,2033
1,4a3ca9315b744ce9f8e9374361493884,1987
2,1f50f920176fa81dab994f9023523100,1931
3,cc419e0650a3c5ba77189a1882b7556a,1775
4,da8622b14eb17ae2831f4ac5b9dab84a,1551


Query 6.2: Top Sellers by Total Revenue

In [21]:
sql = """
SELECT
    oi.seller_id,
    ROUND(SUM(oi.price), 2) AS total_revenue
FROM olist_order_items oi
JOIN olist_orders o ON oi.order_id = o.order_id
WHERE o.order_status = 'delivered'
GROUP BY oi.seller_id
ORDER BY total_revenue DESC
LIMIT 5
"""

df_query6b = pd.read_sql_query(sql, con=engine)
df_query6b.head()


,seller_id,total_revenue
0,4869f7a5dfa277a7dca6462dcf3b52b2,226987.93
1,53243585a1d6dc2643021fd1853d8905,217940.44
2,4a3ca9315b744ce9f8e9374361493884,196882.12
3,fa1c13f2614d7b5c4749cbc52fecda94,190917.14
4,7c67e1448b00f6e969d365cea6b010ab,186570.05


Query 6.3: Top Sellers by Unique Customers

In [22]:
sql = """
SELECT
    oi.seller_id,
    COUNT(DISTINCT o.customer_id) AS customer_count
FROM olist_order_items oi
JOIN olist_orders o ON oi.order_id = o.order_id
WHERE o.order_status = 'delivered'
GROUP BY oi.seller_id
ORDER BY customer_count DESC
LIMIT 5
"""

df_query6c = pd.read_sql_query(sql, con=engine)
df_query6c.head()


,seller_id,customer_count
0,6560211a19b47992c3666cc44a7e94c0,1819
1,4a3ca9315b744ce9f8e9374361493884,1772
2,cc419e0650a3c5ba77189a1882b7556a,1651
3,1f50f920176fa81dab994f9023523100,1399
4,da8622b14eb17ae2831f4ac5b9dab84a,1311


Query 6.4: Top Sellers by Number of 5-Star Reviews

In [23]:
sql = """
SELECT
    oi.seller_id,
    COUNT(DISTINCT oi.order_id) AS five_star_orders
FROM olist_order_items oi
JOIN olist_order_reviews r ON oi.order_id = r.order_id
JOIN olist_orders o ON oi.order_id = o.order_id
WHERE r.review_score = 5
  AND o.order_status = 'delivered'
GROUP BY oi.seller_id
ORDER BY five_star_orders DESC
LIMIT 5
"""

df_query6d = pd.read_sql_query(sql, con=engine)
df_query6d.head()


,seller_id,five_star_orders
0,cc419e0650a3c5ba77189a1882b7556a,993
1,6560211a19b47992c3666cc44a7e94c0,944
2,4a3ca9315b744ce9f8e9374361493884,859
3,1f50f920176fa81dab994f9023523100,841
4,da8622b14eb17ae2831f4ac5b9dab84a,771


Query 7: Delivery Success Rate Across States

In [24]:
sql = """
SELECT
    c.customer_state AS state,
    ROUND(100.0 * SUM(CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END) / COUNT(*), 2) AS delivery_success_rate_pct
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
GROUP BY state
ORDER BY delivery_success_rate_pct DESC
"""

df_query7 = pd.read_sql_query(sql, con=engine)
df_query7.head()


,state,delivery_success_rate_pct
0,AC,98.77
1,AP,98.53
2,ES,98.13
3,MS,98.04
4,AM,97.97


Query 8: Preferred Form of Payment for Each Product Category

In [25]:
sql = """
SELECT category, payment_type, pay_count FROM (
  SELECT
    pt.product_category_name_english AS category,
    p.payment_type,
    COUNT(*) AS pay_count,
    RANK() OVER (
      PARTITION BY pt.product_category_name_english
      ORDER BY COUNT(*) DESC
    ) AS rank
  FROM olist_order_items oi
  JOIN olist_products_dataset prod ON oi.product_id = prod.product_id
  JOIN product_category_name_translation pt ON prod.product_category_name = pt.product_category_name
  JOIN olist_order_payments p ON oi.order_id = p.order_id
  GROUP BY pt.product_category_name_english, p.payment_type
)
WHERE rank = 1
"""

df_query8 = pd.read_sql_query(sql, con=engine)
df_query8.head()


,category,payment_type,pay_count
0,agro_industry_and_commerce,credit_card,145
1,air_conditioning,credit_card,222
2,art,credit_card,153
3,arts_and_craftmanship,credit_card,14
4,audio,credit_card,263


 Query 9: Distance Between Cities

In [26]:
import numpy as np

geo_avg = df_geo.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean'
}).reset_index()

sample_order = df_orders.merge(df_customers, on='customer_id') \
                        .merge(df_order_items, on='order_id') \
                        .merge(df_sellers, on='seller_id') \
                        .head(1)

cust_zip = sample_order['customer_zip_code_prefix'].values[0]
seller_zip = sample_order['seller_zip_code_prefix'].values[0]

cust_coords = geo_avg[geo_avg['geolocation_zip_code_prefix'] == cust_zip]
seller_coords = geo_avg[geo_avg['geolocation_zip_code_prefix'] == seller_zip]


def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Calculate distance
lat1, lon1 = cust_coords.iloc[0]['geolocation_lat'], cust_coords.iloc[0]['geolocation_lng']
lat2, lon2 = seller_coords.iloc[0]['geolocation_lat'], seller_coords.iloc[0]['geolocation_lng']
distance_km = haversine(lat1, lon1, lat2, lon2)

print(f"Distance between customer and seller: {round(distance_km, 2)} km")


Distance between customer and seller: 18.58 km
